<a href="https://colab.research.google.com/github/abubakrsiddq/ImageDehazing/blob/main/models/custom_loss/TestNet_with_new_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import random
from PIL import Image
import time
import datetime

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.optimizers import Adam




from math import log10, sqrt
import cv2
from skimage.metrics import structural_similarity as ssim



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, size = (412, 548), antialias = True)
    img = img / 255.0
    return img

In [ ]:
def dataset_preposses(orig_path='/content/drive/MyDrive/dataset/clear_images',haze_path='/content/drive/MyDrive/dataset/haze',percentage=0.3,validation_size=64,test_size=64,seed_val=101):
  '''
  parameters:
  orig_path(string): path of ground truth folder
  haze_path(string): path of haze folder
  percentage(float): percentage of dataset to load
  validation_size(int): the no. of validation images
  test_size(int): the no. of test images

  returns:
  haze_list,validation_list,test_list
  '''
  random.seed(seed_val)
  pth=haze_path+'/*.jpg'
  haze_path_list = glob.glob(pth)
  orig_path_list=glob.glob(orig_path+'/*.jpg')
  #print(orig_path_list)
  random.shuffle(haze_path_list)
  #print(haze_path_list)
  haze_path_dict={}
  haze_count_dict={}
  haze_list=[]
  no_per_set=int(percentage*35)
  for i in haze_path_list:
    name=i.split('/')[-1].split('_')[0]
    if(int(name)>468):
      try:
        if(haze_count_dict[name]<no_per_set):
          haze_path_dict[name].append(i)
          
          haze_count_dict[name]+=1;
          
      except KeyError:
       
        haze_path_dict[name]=[]
        haze_path_dict[name].append(i)
        haze_count_dict[name]=1
    #print(haze_path_dict)
  test_list=haze_path_list[-1*test_size:]
  val_list=haze_path_list[-1*(validation_size+test_size):-1*test_size];

  for (key,val) in haze_path_dict.items():
    for i in val:
      haze_list.append(i)
  return haze_list,val_list,test_list


def gen_dataset(ar):
  '''
  parameters
  list of paths
  return
  list with gt attached 
  '''
  orig_path='/content/drive/MyDrive/dataset/clear_images'
  haze_pth='/content/drive/MyDrive/dataset/haze'
  lst=[]
  for i in ar:
    name=i.split('/')[-1].split('_')[0]
    pthlist=[i,orig_path+'/'+name+'.jpg']
    lst.append(pthlist)
  return lst

def data_path(orig_img_path = './drive/MyDrive/reside/archive/clear_images', hazy_img_path = './drive/MyDrive/reside/archive/haze'):
  
  (a,b,c)=dataset_preposses(orig_path=orig_img_path,haze_path=hazy_img_path)
  a=gen_dataset(a)
  b=gen_dataset(b)
  return a,b



In [ ]:
def dataloader(train_data, val_data, batch_size):
    
    train_data_orig = tf.data.Dataset.from_tensor_slices([img[1] for img in train_data]).map(lambda x: load_image(x))
    train_data_haze = tf.data.Dataset.from_tensor_slices([img[0] for img in train_data]).map(lambda x: load_image(x))
    train = tf.data.Dataset.zip((train_data_haze, train_data_orig)).shuffle(buffer_size=100).batch(batch_size)
    
    val_data_orig = tf.data.Dataset.from_tensor_slices([img[1] for img in val_data]).map(lambda x: load_image(x))
    val_data_haze = tf.data.Dataset.from_tensor_slices([img[0] for img in val_data]).map(lambda x: load_image(x))
    val = tf.data.Dataset.zip((val_data_haze, val_data_orig)).shuffle(buffer_size=100).batch(batch_size)
    
    return train, val

In [ ]:
def display_img(model, hazy_img, orig_img):
    
    dehazed_img = model(hazy_img, training = True)
    plt.figure(figsize = (15,15))
    
    display_list = [hazy_img[0], orig_img[0], dehazed_img[0]]
    title = ['Hazy Image', 'Ground Truth', 'Dehazed Image']
    
    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
        
    plt.show()

# **network**

In [ ]:


def  custom_Loss():
    
    inputs = tf.keras.Input(shape = [412,548, 3])     # height, width of input image changed because of error in output
    conv = Conv2D(filters = 50, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu',)(inputs)
    poolLayer=AveragePooling2D(pool_size=(2,2))(conv)
    conv1 = Conv2D(filters = 50, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu')(poolLayer)
    poolLayer=AveragePooling2D(pool_size=(2,2))(conv1)  
    #flat=Flatten()(poolLayer)
    dens1=Dense(10,activation='relu')(poolLayer)
    dens2=Dense(10,activation='relu')(dens1)
    deconv1=Conv2DTranspose(50,kernel_size=(3,3),padding='same',activation='relu')(dens2)
    upsamp1=UpSampling2D(size=(2,2))(deconv1)
    deconv2=Conv2DTranspose(50,kernel_size=(3,3),padding='same',activation='relu')(upsamp1)
    upsamp2=UpSampling2D(size=(2,2))(deconv2)
    deconv3=Conv2DTranspose(3,kernel_size=(3,3),padding='same',activation='linear')(upsamp2)
    output = deconv3
    
    return Model(inputs = inputs, outputs = output)



In [ ]:
model= custom_Loss()
model.build([412,548,3])
model.summary()

In [ ]:
# Hyperparameters
epochs = 10
batch_size = 8


#train_data, val_data = data_path(orig_img_path = './drive/MyDrive/reside/archive/clear_images', hazy_img_path = './drive/MyDrive/reside/archive/haze')
train_data, val_data = data_path(orig_img_path = './drive/MyDrive/dataset/clear_images', hazy_img_path = './drive/MyDrive/dataset/haze')
train, val = dataloader(train_data, val_data, batch_size)

optimizer = Adam(learning_rate = 1e-4)
net = custom_Loss()
#net= tf.keras.models.load_model('/content/drive/MyDrive/nets/test_custom_loss_net',compile=False)
train_loss_tracker = tf.keras.metrics.MeanSquaredError(name = "train loss")
val_loss_tracker = tf.keras.metrics.MeanSquaredError(name = "val loss")

In [ ]:
def train_model(epochs, train, val, net, train_loss_tracker, val_loss_tracker, optimizer):
    
    for epoch in range(epochs):
        
        print("\nStart of epoch %d" % (epoch,), end=' ')
        start_time_epoch = time.time()
        start_time_step = time.time()
        
        # training loop
        
        for step, (train_batch_haze, train_batch_orig) in enumerate(train):

            with tf.GradientTape() as tape:

                train_logits = net(train_batch_haze, training = True)
                loss = mean_squared_error(train_batch_orig, train_logits)

            grads = tape.gradient(loss, net.trainable_weights)
            optimizer.apply_gradients(zip(grads, net.trainable_weights))

            train_loss_tracker.update_state(train_batch_orig, train_logits)
            if step == 0:
                print('[', end='')
            if step % 64 == 0:
                print('=', end='')
        
        print(']', end='')
        print('  -  ', end='')
        print('Training Loss: %.4f' % (train_loss_tracker.result()), end='')
        
        # validation loop
        
        for step, (val_batch_haze, val_batch_orig) in enumerate(val):
            val_logits = net(val_batch_haze, training = False)
            val_loss_tracker.update_state(val_batch_orig, val_logits)
            
            if step % 32 ==0:
                display_img(net, val_batch_haze, val_batch_orig)
        
        print('  -  ', end='')
        print('Validation Loss: %.4f' % (val_loss_tracker.result()), end='')
        print('  -  ', end=' ')
        print("Time taken: %.2fs" % (time.time() - start_time_epoch))
        
        net.save('trained_model')           # save the model(variables, weights, etc)
        train_loss_tracker.reset_states()
        val_loss_tracker.reset_states()



In [ ]:


%%time
train_model(epochs, train, val, net, train_loss_tracker, val_loss_tracker, optimizer)



In [ ]:
net.save('./drive/MyDrive/nets/test_custom_loss_net')
model=net
#model.build([412,548,3])
#model.summary()

In [ ]:
import cv2
import matplotlib.pyplot as plt

def msee(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err
def compare_images(imageA, imageB):
  # compute the mean squared error and structural similarity
  # index for the images
  m = msee(imageA, imageB)
  s = ssim(imageA, imageB,multichannel=True)
  return s
  #s=tf.image.ssim(imageA, imageB, max_val=255, filter_size=11,filter_sigma=1.5, k1=0.01, k2=0.03)
def find_psnr(imageA,imageB):
   return cv2.PSNR(imageA,imageB)

def evaluate_gen(net):
    
    #test_img = glob.glob(test_img_path +'/*.jpg')
    test_img=glob.glob('/content/drive/MyDrive/Final_compare/HAZY/*.jpg')
    
    #random.shuffle(test_img)
    i=1;
    for img in test_img:
        
        img = tf.io.read_file(img)
        img = tf.io.decode_jpeg(img, channels = 3)
        
        img = tf.image.resize(img, size = (412,548), antialias = True)
        
        img = img / 255.0
        print(i,end=" ")
        img = tf.expand_dims(img, axis = 0)      #transform input image from 3D to 4D ###
        
        dehaze = net(img)
        dehaze=tf.image.resize(dehaze, size = (413,550), antialias = True)
        #plt.figure(figsize = (80, 80))
        
        #display_list = [img[0], dehaze[0]]       #make the first dimension zero
        im=dehaze[0]
        directory = '/content/drive/MyDrive/Final_compare/test_custom_loss_net'
        
        os.chdir(directory)
        filename = str(i) + '_outdoor_gen.jpg'
        #print(filename)
        #cv2.imwrite(filename,im) 
        #plt.imsave(filename,im)
        tf.keras.preprocessing.image.save_img(filename,im)

        os.chdir('/content')
        i+=1;
    orig_img_path='/content/drive/MyDrive/Final_compare/GT'
    orig_img = glob.glob(orig_img_path + '/*.jpg')
    orig_img.sort()

    test_img_path='/content/drive/MyDrive/Final_compare/test_custom_loss_net'
    
    test_img = glob.glob(test_img_path + '/*.jpg')
    test_img.sort()


    psnr_val=0;ssim_val=0;
    n=len(orig_img)
    for i in range(len(orig_img)):
      os.chdir(orig_img_path)
      original = cv2.imread(orig_img[i]) 
      #original=tf.image.resize_with_crop_or_pad(original, 400, 520)
      original = original[3:-4,3:-4,]
      os.chdir(test_img_path)
      hazy = cv2.imread(test_img[i]) 
      #hazy=tf.image.resize_with_crop_or_pad(hazy, 400, 520)
      hazy=hazy[3:-4,3:-4]
      psnr_val+=find_psnr(original, hazy)
      ssim_val+=compare_images(original, hazy)
      #print(i,end=" ")
    total_psnr=psnr_val/n
    total_ssim=ssim_val/n
    print("\n psnr=",total_psnr)
    print("ssim=",total_ssim)



In [ ]:


new_model = tf.keras.models.load_model('/content/drive/MyDrive/nets/test_custom_loss_net',compile=False)


evaluate_gen(new_model)

